In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os, sys
%matplotlib inline

os.chdir('..\..\..\..')

from input_configuration import base_year

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
working_dir = r'outputs/agg/dash'
# pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.2%}".format

In [3]:
def load_data(fname):
    """ open and join model and survey data """

    df_model = pd.read_csv(os.path.join(working_dir,fname+'.csv'))
    df_model['source'] = 'model'
    df_survey = pd.read_csv(os.path.join(working_dir,'survey',fname+'.csv'))
    df_survey['source'] = 'survey'
    df = df_model.append(df_survey)

    return df

In [4]:
parcel_geog = pd.read_sql_table('parcel_'+base_year+'_geography', 'sqlite:///inputs/db/soundcast_inputs.db',
        columns=['CountyName','ParcelID'])

In [5]:
def dist_by_mode(purpose):
    df = load_data('tour_distance')
    # Trim any unreasonable survey distances off by taking max distance as max modeled bin value
    max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
    df = df.loc[df['tautodist_bin'] <= max_model_dist]
    df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
    df_dist = pd.DataFrame()
    for source in ['model','survey']:
        _df = df[(df['source'] == source) & (df['pdpurp'] == purpose)]
        df1 = _df.groupby('tmodetp').sum()[['wt_autodist']].reset_index()
        df2 = _df.groupby('tmodetp').sum()[['toexpfac']].reset_index ()
        _df = df1.merge(df2, on='tmodetp')
        _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
        _df['source'] = source
        df_dist = df_dist.append(_df)
    df = df_dist.pivot_table(index='tmodetp', columns='source', aggfunc='first', values='commute_distance')
    df.rename(columns={'tmodetp': 'Tour Mode'}, inplace=True)
    for col in ['School Bus',' Park']:
        if (col in df.index) & (purpose != 'School'):
            df.drop(col, axis=0, inplace=True)
        if (col != 'School Bus') & (col in df.index):
            df.drop(col, axis=0, inplace=True)
    
    return df

### Total

In [6]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.667215
survey,4.277133


### By Purpose

In [7]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df

source,model,survey
pdpurp,,
Change Mode Inserted Purpose,NaN,15.381360
Escort,6.998221,4.346792
Meal,2.817477,4.978618
Personal Business,6.765277,5.431595
School,3.667215,4.277133
Shop,4.733719,4.908592
Social,4.470941,5.251707
Work,11.439409,10.979529


# Tour Distance


### Average Work Tour Distance By Mode

In [80]:
dist_by_mode('Work')

source,model,survey
tmodetp,,
Bike,5.565858,4.588920
HOV2,11.469063,10.330636
HOV3+,11.841534,14.394094
Park,15.785402,27.611739
SOV,12.022697,11.008877
TNC,6.006925,4.475837
Transit,12.911899,11.022154
Walk,0.762817,1.693188


### Average School Tour Distance by Mode

In [56]:
dist_by_mode('School')

source,model,survey
tmodetp,,
Bike,2.569636,1.537459
HOV2,3.522687,4.754287
HOV3+,3.642029,4.325446
SOV,7.355436,5.249002
School Bus,3.193878,2.862676
TNC,2.332774,3.352090
Transit,5.969371,6.589889
Walk,1.112619,1.041474


### Average Personal Business Tour Distance by Mode

In [59]:
dist_by_mode('Personal Business')

source,model,survey
tmodetp,,
Bike,3.265017,3.085207
HOV2,7.823077,6.325368
HOV3+,7.781722,5.201211
SOV,6.614503,5.965793
TNC,1.995497,5.025672
Transit,6.696441,5.358596
Walk,1.182023,1.707953


### Average Shopping Tour Distance by Mode

In [64]:
dist_by_mode('Shop')

source,model,survey
tmodetp,,
Bike,2.408022,0.558925
HOV2,5.474640,5.238435
HOV3+,5.629333,5.980384
SOV,4.688517,4.703844
TNC,1.472745,2.107666
Transit,4.368060,2.653354
Walk,1.084966,1.247437


### Average Meal Tour Distance by Mode

In [66]:
dist_by_mode('Meal')

source,model,survey
tmodetp,,
Bike,1.080935,3.691325
HOV2,4.645685,4.827027
HOV3+,4.978334,7.176759
SOV,2.804167,5.496125
TNC,1.208114,2.311416
Transit,3.070070,7.437024
Walk,0.686159,1.840626


### Average Social Tour Distance by Mode

In [67]:
dist_by_mode('Social')

source,model,survey
tmodetp,,
Bike,2.479569,2.555915
HOV2,6.010500,7.431229
HOV3+,6.003816,5.063787
SOV,5.339749,6.037328
TNC,1.412438,2.943097
Transit,5.090444,9.010155
Walk,0.953371,2.517812


### Average Escort Tour Distance by Mode

In [76]:
dist_by_mode('Escort')

source,model,survey
tmodetp,,
Bike,12.383043,0.694587
HOV2,6.493710,4.732855
HOV3+,7.078159,4.089530
SOV,21.570943,5.742623
TNC,1.602665,0.500000
Transit,2.221212,1.802905
Walk,0.958653,1.860571


Survey Sample Size Reference by Mode and Purpose

In [9]:
# Survey Sample size
df_tour_survey = pd.read_csv(r'inputs/base_year/survey/_tour.tsv', delim_whitespace=True)

In [10]:
df_tour_survey.pivot_table(index='tmodetp',columns='pdpurp', values='toexpfac', aggfunc='count')

pdpurp,1,2,3,4,5,6,7,10
tmodetp,,,,,,,,
1,1936.0,377.0,329.0,647.0,979.0,1384.0,2736.0,NaN
2,387.0,60.0,10.0,23.0,23.0,32.0,96.0,NaN
3,6358.0,232.0,77.0,1638.0,1320.0,687.0,1635.0,1.0
4,1957.0,524.0,1309.0,956.0,979.0,586.0,1292.0,NaN
5,919.0,813.0,901.0,614.0,479.0,475.0,1210.0,1.0
6,4965.0,493.0,58.0,468.0,202.0,246.0,598.0,4.0
7,161.0,2.0,NaN,8.0,1.0,7.0,15.0,NaN
8,17.0,431.0,1.0,66.0,NaN,NaN,14.0,NaN
9,271.0,18.0,1.0,55.0,13.0,72.0,67.0,NaN
